In [10]:
import quantfreedom as qf
import numpy as np
import pandas as pd

from quantfreedom.backtester.indicators.talib_ind import from_talib
from quantfreedom._typing import (
    pdFrame, pdSeries
)
prices = pd.read_csv('30min.csv', index_col='time')
rsi_ind = from_talib(
    func_name='rsi',
    df_prices=prices,
    cart_product=False,
    combos=False,
    timeperiod=[15, 20],
)


In [9]:
tester = 'asd'
if tester.lower() in ('open', 'high', 'low', 'close'):
    print('yes')

In [11]:
np.zeros((2,5))

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [57]:
temp_bigger_df_array = np.asarray(np.random.random_integers(0,1, size=(10,6)),dtype=np.bool_)
temp_smaller_df_array = np.asarray(np.random.random_integers(0,1, size=(10,6)),dtype=np.bool_)
combine_array = np.full((10,6), 8)

In [58]:
combine_array[:, 0] = np.logical_and(
                temp_bigger_df_array[:,0] == True,
                temp_smaller_df_array[:, 0] == True)
combine_array

array([[1, 8, 8, 8, 8, 8],
       [0, 8, 8, 8, 8, 8],
       [0, 8, 8, 8, 8, 8],
       [1, 8, 8, 8, 8, 8],
       [1, 8, 8, 8, 8, 8],
       [0, 8, 8, 8, 8, 8],
       [0, 8, 8, 8, 8, 8],
       [0, 8, 8, 8, 8, 8],
       [0, 8, 8, 8, 8, 8],
       [0, 8, 8, 8, 8, 8]])

In [59]:
temp_bigger_df_array[:,0]

array([ True, False, False,  True,  True, False,  True, False, False,
        True])

In [60]:
temp_smaller_df_array[:, 0]

array([ True,  True, False,  True,  True,  True, False, False, False,
       False])